In [1]:
import torch
import numpy as np
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [25]:
import torch.nn as nn

class Gen(nn.Module):
  # GAN takes an input of noise of size 10 and constructs a 7 bit number in the form of a 7 element tensor
  def __init__(self):
    super(Gen, self).__init__()
    self.net = nn.Sequential(
        nn.Linear(10,10),
        nn.ReLU(),
        nn.Linear(10,7),
        nn.Sigmoid()
    )
  def forward(self, input):
    return self.net(input)

In [118]:
class Discrim(nn.Module):
  def __init__(self):
    super(Discrim, self).__init__()
    self.net = nn.Sequential(
        #nn.Linear(7,7),
        #nn.ReLU(),
        nn.Linear(7,1),
        #nn.ReLU()
    )
  def forward(self, input):
    return self.net(input)

In [ ]:
import random
def generate_real(num):
  inputs = [[int(x) for x in format(2*random.randint(0,63), "07b")] for x in range(num)]
  
  return torch.tensor(inputs)

generate_real(20)

In [16]:
# https://discuss.pytorch.org/t/set-constraints-on-parameters-or-layers/23620/2
class WeightClipper(object):
    def __init__(self, c, frequency=5):
        self.frequency = frequency
        self.c = abs(c)

    def __call__(self, module):
        # filter the variables to get the ones you want
        if hasattr(module, 'weight'):
            w = module.weight.data
            w = w.clamp(-self.c,self.c)
            module.weight.data = w

In [144]:
# https://agustinus.kristia.de/techblog/2017/02/04/wasserstein-gan/
# hyperparameters from tutorial
# reimplement training loop
# train discrim several times, then train generator

from torch import optim

alpha = 0.0005 # learning rate
c = 0.1 # clamp discriminator weights
n_critic = 5 # number of iterations of training for discriminator

generator = Gen()
discriminator = Discrim()

clipper = WeightClipper(c)

one = torch.FloatTensor([1])
m_one = torch.FloatTensor([-1])

def train(num_epochs):
    gen_optim = optim.RMSprop(generator.parameters(), lr = alpha)
    discrim_optim = optim.RMSprop(discriminator.parameters(), lr = alpha)

    for i in range(num_epochs):
        # training discriminator
        for j in range(n_critic):
            noise = torch.rand(200, 10).float()
            fake = generator.forward(noise)
            real = generate_real(200).float()
            fake_loss = discriminator.forward(fake.detach())
            real_loss = discriminator.forward(real)
            # wasserstein loss = real - fake, goal is to maximize, equivalent to minimizing negative
            discrim_optim.zero_grad()
            discrim_loss = -(torch.mean(real_loss, dim=0) - torch.mean(fake_loss, dim = 0))
            discrim_loss.backward()
            discrim_optim.step()
            discriminator.apply(clipper)
        # training generator
        noise = torch.rand(200,10).float()
        gen_optim.zero_grad()
        gen_loss = -torch.mean(discriminator.forward(generator.forward(noise)), dim = 0)
        gen_loss.backward()
        gen_optim.step()
  
train(10000)

In [151]:
test_input = torch.rand(1,10)
print("random input:", test_input.numpy())
output = torch.flatten(generator.forward(test_input).detach()).numpy()
print("output:",output)
total = 0
for i in range(7):
  total += 2**(6-i) * output[i]
print("generated number:", total)

random input: [[0.42171234 0.9290839  0.01743454 0.5376518  0.6288203  0.47541106
  0.35840267 0.45026326 0.8602835  0.41878998]]
output: [5.3042114e-01 4.7890449e-01 5.3126240e-01 4.5517012e-01 4.9667582e-01
 5.0282925e-01 3.2930450e-11]
generated number: 64.40581750873044


In [ ]:
"""
Results:


example output:
[5.3427917e-01 4.6246558e-01 5.0097013e-01 4.8601577e-01 4.9112451e-01 5.1165754e-01 4.5578025e-10] => 63.88

first 6 entries approximately 0.5 each, last entry is 0

All valid inputs are 7 bit binary numbers. First 6 entries described by Bernoulli(0.5), last entry always 0

Generator learns distribution of valid inputs and converges to the expected value
"""